In [49]:
import sys
import numpy as np
sys.path.append('../')
from model import data_preparation as prep

In [62]:
con = prep.create_connection()
prepared_data = prep.prepare_data(con)

In [63]:
prepared_data

{257: {'abahn': True,
  'brand': 'shell',
  'bstr': True,
  'county': '01055',
  'sstr': False,
  'state': 'schleswig-holstein',
  'time_series': {'test': array([1379, 1379, 1379, ..., 1349, 1489, 1489]),
   'train': array([1399, 1399, 1399, ..., 1379, 1379, 1379])}},
 569: {'abahn': True,
  'brand': 'hoyer',
  'bstr': True,
  'county': '13072',
  'sstr': False,
  'state': 'mecklenburg-vorpommern',
  'time_series': {'test': array([1299, 1299, 1299, ..., 1289, 1289, 1399]),
   'train': array([1339, 1339, 1339, ..., 1339, 1299, 1299])}},
 862: {'abahn': False,
  'brand': 'aral',
  'bstr': True,
  'county': '020000000000',
  'sstr': True,
  'state': 'hamburg',
  'time_series': {'test': array([1329, 1329, 1329, ..., 1329, 1489, 1489]),
   'train': array([1319, 1319, 1319, ..., 1349, 1349, 1329])}},
 2720: {'abahn': False,
  'brand': 'raiffeisen',
  'bstr': True,
  'county': '03456',
  'sstr': False,
  'state': 'niedersachsen',
  'time_series': {'test': array([1399, 1399, 1399, ..., 1359, 1

In [54]:
for key in prepared_data:
    prepared_data[key]["time_series"]["test"] = np.log(prepared_data[key]["time_series"]["test"].reshape(-1,1))
    prepared_data[key]["time_series"]["train"] = np.log(prepared_data[key]["time_series"]["train"].reshape(-1,1))
    

In [56]:
from statsmodels.tsa.stattools import pacf
x_pacf=pacf( prepared_data[593]['time_series']['train'] , nlags =10, method='ols')

In [57]:
x_pacf

array([ 1.        ,  0.93359773, -0.03709959, -0.07266407, -0.04372252,
       -0.04391209, -0.03247097, -0.00742075,  0.22071472,  0.18765126,
        0.08870031])

In [58]:

from nnet_ts import *
count = 0
ahead =12

In [59]:
def train_net_on_station(trainset, ahead=12, count = 0):
    x = trainset
    x=x.reshape ( -1)
    while(count < ahead):
        end =len(x)-ahead+count
        np.random.seed(2016)
        fit1 = TimeSeriesNnet(hidden_layers =[7,3],activation_functions = [ "tanh" , "tanh" ] )
        fit1.fit(x[0:end],lag =1,epochs = 100)


        out = fit1.predict_ahead(n_ahead = 1)
        print ( " Obs : " , count +1, " x = " , round( x[count] , 4 ) , " prediction =", pd.Series(out))

        pred.append ( out )
        count = count + 1
        return fit1
    
train_net_on_station(prepared_data[593]['time_series']['train'])

INFO:Building regressor matrix
INFO:Scaling data
INFO:Checking network consistency
INFO:Training neural net
/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


KeyboardInterrupt: 

In [65]:
def encodeState(data):
    states = {
     'baden-württemberg':1,
     'bayern':2,
     'berlin':3,
     'brandenburg':4,
     'bremen':5,
     'hamburg':6,
     'hessen':7,
     'mecklenburg-vorpommern':8,
     'niedersachsen':9,
     'nordrhein-westfalen':10,
     'rheinland-pfalz':11,
     'saarland':12,
     'sachsen':13,
     'sachsen-anhalt':14,
     'schleswig-holstein':15,
     'thüringen':16
    }
    data = states[data["state"]]
    return data

prepared_data[257]= encodeState(prepared_data[257])


KeyError: 15

In [69]:
x_train = prepared_data[257]["time_series"]["train"][0:len(prepared_data[257]["time_series"]["train"])-1]
y_train = prepared_data[257]["time_series"]["train"][1:len(prepared_data[257]["time_series"]["train"])]
x_test = prepared_data[257]["time_series"]["test"][0:len(prepared_data[257]["time_series"]["test"])-1]
y_test = prepared_data[257]["time_series"]["test"][1:len(prepared_data[257]["time_series"]["test"])]


In [73]:
x_train_adjusted = []
for key in x_train:
    x_train_adjusted[key] = [key,prepared_data[257]["state"]]

IndexError: list assignment index out of range